In [ ]:
!pip install transformers accelerate git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-7v216d1z
  Running command git clone -q https://github.com/huggingface/accelerate /tmp/pip-req-build-7v216d1z
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.9 MB 9.9 MB/s 
     |████████████████████████████████| 143 kB 82.1 MB/s 
     |████████████████████████████████| 6.6 MB 82.7 MB/s 
     |████████████████████████████████| 163 kB 82.6 MB/s 
  Created wheel for accelerate: filename=accelerate-0.13.0.dev0-py3-none-any.whl size=148603 sha256=f796d65fce70284e7d1eadedafa0b76a9772aaf5e6d527c2c3e34ee7c54548fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-7ri_cg8u/wheels/81/c1/23/6068c1115888b4dd7da88f966c002c30840985c047f6cc1653
Successfully built accelerate


In [ ]:
from transformers import pipeline, set_seed, AutoTokenizer
import numpy as np
import pandas as pd
import random
from datetime import datetime
import os

In [ ]:
model_name = 'gpt2' #'gpt2-large' 'gpt2-xl'

experiment_cycles = 1000

num_prime_tokens = 64
num_probe_tokens = 10

top_k = 5

prime_pause_str = '.' # (Punctuation) token (string) that separates trigrams in prime input

cols = ['Setting','TkInp','TkOut1','TkOut2','TkOut3','TkOut4','TkOut5','Inp','Out1','Out2','Out3','Out4','Out5']

#'w' and 'y' excluded from both, 'q' excluded from consonants
vowels = np.array(['a','e','i','o','u','A','E','I','O','U'])
consonants = np.array(['b','c','d','f','g','h','j','k','l','m',
                       'n','p','r','s','t','v','x','z','B','C','D','F','G','H','J','K','L','M','N','P','R','S','T','V','X','Z'])

tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = pipeline('text-generation', model=model_name, device_map='auto')#device=0)
set_seed(42) # Since the generation relies on some randomness, we set a seed for reproducibility:

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

# Run Experiment

In [ ]:
pauses = np.repeat(tokenizer.convert_tokens_to_ids(prime_pause_str), num_prime_tokens)
prime_order = np.arange(num_prime_tokens)
np.random.shuffle(prime_order)

# Restricted Syllables: Preparation
v_str = str(vowels).replace("['", "(").replace("']", ")").replace("' '", "|")
c_str = str(consonants).replace("['", "(").replace("']", ")").replace("' '", "|")

tkns = []
for i in range(len(tokenizer.vocab.keys())): tkns.append(tokenizer.decode([i]))
df = pd.DataFrame(data={'Token':tkns})

syl_df = pd.DataFrame(data={'Token':[]})
syl_df = syl_df.append(df[df.Token.str.match(f'^ {v_str}{c_str}$')]) #_VC
syl_df = syl_df.append(df[df.Token.str.match(f'^ {c_str}{v_str}$')]) #_CV
syl_df = syl_df.sort_index()
syl_df.reset_index(inplace=True)
syl_df = syl_df.rename(columns={'index':'TokenID'})
syls = syl_df.TokenID.to_numpy()


def token_selector(num_tkns, ids_xcl=[]):
  ids = random.sample(set(syls)-set(ids_xcl), num_tkns)
  return ids

def output_generation(prime_ASR, probe_a=None):
  temp = [[],[],[],[],[],[],[],[],[],[],[],[],[]]
  
  if probe_a == None: 
    temp[0] = 'no-probe'
    prime_input = prime_input_non = tokenizer.decode(prime_ASR.flatten())
    max_new=3
  else: 
    temp[0] = 'probe A'
    prime_input = tokenizer.decode(np.append(prime_ASR.flatten(), probe_a))
    prime_input_non = tokenizer.decode(prime_ASR.flatten())
    max_new=2

  output = generator(prime_input, max_new_tokens=max_new, max_length=None, num_return_sequences=top_k)
  #output = generator(prime_input, max_length=num_prime_tokens*4+3, num_return_sequences=top_k)

  if prime_input_non == prime_input: temp[0] = 'no-probe' 
  else: temp[0] = 'probe A'
  temp[1] = tokenizer.encode(prime_input)
  temp[2] = tokenizer.encode(output[0]['generated_text'])[len(temp[1])-(3-max_new):]
  temp[3] = tokenizer.encode(output[1]['generated_text'])[len(temp[1])-(3-max_new):]
  temp[4] = tokenizer.encode(output[2]['generated_text'])[len(temp[1])-(3-max_new):]
  temp[5] = tokenizer.encode(output[3]['generated_text'])[len(temp[1])-(3-max_new):]
  temp[6] = tokenizer.encode(output[4]['generated_text'])[len(temp[1])-(3-max_new):]
  temp[7] = prime_input
  temp[8] = output[0]['generated_text'].replace(prime_input_non,'')
  temp[9] = output[1]['generated_text'].replace(prime_input_non,'')
  temp[10] = output[2]['generated_text'].replace(prime_input_non,'')
  temp[11] = output[3]['generated_text'].replace(prime_input_non,'')
  temp[12] = output[4]['generated_text'].replace(prime_input_non,'')

  return pd.DataFrame(temp,index=cols).T

df_AAB_priming = pd.DataFrame(columns=cols)
df_ABA_priming = pd.DataFrame(columns=cols)
df_ABB_priming = pd.DataFrame(columns=cols)
df_ABCa_priming = pd.DataFrame(columns=cols)
df_ABCb_priming = pd.DataFrame(columns=cols)

for i_exp in range(experiment_cycles):
  prime_AAB = prime_ABA = prime_ABB = np.array([]).astype('int64')

  ids_prime_a = token_selector(num_prime_tokens)
  ids_prime_b = token_selector(num_prime_tokens,ids_prime_a)

  combs_AB = np.transpose((ids_prime_a, ids_prime_b))

  prime_AAB = np.insert(combs_AB[:,(0,0,1)],3,pauses,axis=1)
  prime_ABA = np.insert(combs_AB[:,(0,1,0)],3,pauses,axis=1)
  prime_ABB = np.insert(combs_AB[:,(0,1,1)],3,pauses,axis=1)
  prime_ABCa = np.insert(np.insert(combs_AB, 2, np.roll(combs_AB[:,0],len(combs_AB[:,0])//2), axis=1),3,pauses,axis=1)[prime_order] #ABCb
  prime_ABCb = np.insert(np.insert(combs_AB, 2, np.roll(combs_AB[:,1],len(combs_AB[:,1])//2), axis=1),3,pauses,axis=1)[prime_order] #ABCb

  ids_probe_a = token_selector(num_probe_tokens,np.concatenate((ids_prime_a, ids_prime_b), axis=0))
  
  df_AAB_priming = pd.concat([df_AAB_priming, output_generation(prime_AAB)], ignore_index=True)#AAB priming
  df_ABA_priming = pd.concat([df_ABA_priming, output_generation(prime_ABA)], ignore_index=True)#ABA priming
  df_ABB_priming = pd.concat([df_ABB_priming, output_generation(prime_ABB)], ignore_index=True)#ABB priming
  df_ABCa_priming = pd.concat([df_ABCa_priming, output_generation(prime_ABCa)], ignore_index=True)#ABB priming
  df_ABCb_priming = pd.concat([df_ABCb_priming, output_generation(prime_ABCb)], ignore_index=True)#ABB priming

  if i_exp%num_probe_tokens == 0:
    for a_probe in ids_probe_a: 
      df_AAB_priming = pd.concat([df_AAB_priming, output_generation(prime_AAB, a_probe)], ignore_index=True)#AAB priming
      df_ABA_priming = pd.concat([df_ABA_priming, output_generation(prime_ABA, a_probe)], ignore_index=True)#ABA priming
      df_ABB_priming = pd.concat([df_ABB_priming, output_generation(prime_ABB, a_probe)], ignore_index=True)#ABB priming
      df_ABCa_priming = pd.concat([df_ABCa_priming, output_generation(prime_ABCa, a_probe)], ignore_index=True)#ABB priming
      df_ABCb_priming = pd.concat([df_ABCb_priming, output_generation(prime_ABCb, a_probe)], ignore_index=True)#ABB priming

  print(f'{i_exp+1} done')


folder = f'{model_name}_{experiment_cycles}_{num_prime_tokens}_{num_probe_tokens}_{top_k}/'
try: os.mkdir(folder)
except FileExistsError: folder = folder

time_st = str(datetime.now())

df_AAB_priming.to_csv(f'{folder}{time_st}_generate_AAB.csv',sep=';')
df_ABA_priming.to_csv(f'{folder}{time_st}_generate_ABA.csv',sep=';')
df_ABB_priming.to_csv(f'{folder}{time_st}_generate_ABB.csv',sep=';')
df_ABCa_priming.to_csv(f'{folder}{time_st}_generate_ABCa.csv',sep=';')
df_ABCb_priming.to_csv(f'{folder}{time_st}_generate_ABCb.csv',sep=';')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

1 done


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2 done


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

998 done


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


999 done


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1000 done


32

# Download

In [ ]:
!zip -r downBL.zip /content/gpt2-medium_1000_64_20_5

updating: content/gpt2-medium_1000_64_20_5/ (stored 0%)
updating: content/gpt2-medium_1000_64_20_5/2022-09-14 20:52:52.256905_generate_ABA.csv (deflated 78%)
updating: content/gpt2-medium_1000_64_20_5/2022-09-14 20:52:52.256905_generate_ABCa.csv (deflated 78%)
updating: content/gpt2-medium_1000_64_20_5/2022-09-14 20:52:52.256905_generate_ABB.csv (deflated 82%)
updating: content/gpt2-medium_1000_64_20_5/2022-09-14 20:52:52.256905_generate_ABCb.csv (deflated 78%)
updating: content/gpt2-medium_1000_64_20_5/2022-09-14 20:52:52.256905_generate_AAB.csv (deflated 82%)


In [ ]:
from google.colab import files
files.download('downBL.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r downBL-l.zip /content/gpt2-medium_1000_64_10_5
from google.colab import files
files.download('downBL-l.zip')

  adding: content/gpt2-medium_1000_64_10_5/ (stored 0%)
  adding: content/gpt2-medium_1000_64_10_5/2022-09-14 23:43:04.876274_generate_ABCb.csv (deflated 78%)
  adding: content/gpt2-medium_1000_64_10_5/2022-09-14 23:43:04.876274_generate_AAB.csv (deflated 82%)
  adding: content/gpt2-medium_1000_64_10_5/2022-09-14 23:43:04.876274_generate_ABCa.csv (deflated 78%)
  adding: content/gpt2-medium_1000_64_10_5/2022-09-14 23:43:04.876274_generate_ABA.csv (deflated 78%)
  adding: content/gpt2-medium_1000_64_10_5/2022-09-14 23:43:04.876274_generate_ABB.csv (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>